# Сделать классификацию данных fakenews
Используя ноутбук занятия (также размещен в папке Materials) и данные fakenews, 3 раза разными способами получить на задаче классификации значение f1 выше 0.91 для методов на sklearn и выше 0.52 для методов на pytorch.

In [1]:
!wget https://raw.githubusercontent.com/diptamath/covid_fake_news/main/data/Constraint_Train.csv

--2022-06-05 10:09:30--  https://raw.githubusercontent.com/diptamath/covid_fake_news/main/data/Constraint_Train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1253562 (1.2M) [text/plain]
Saving to: ‘Constraint_Train.csv’

Constraint_Train.cs 100%[===================>]   1.20M  --.-KB/s    in 0.01s   

2022-06-05 10:09:30 (123 MB/s) - ‘Constraint_Train.csv’ saved [1253562/1253562]



In [2]:
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from tqdm import tqdm
import nltk
nltk.download('punkt')
from gensim.models.word2vec import Word2Vec

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
df = pd.read_csv('Constraint_Train.csv')
df.head(10)

,id,tweet,label
0,1,The CDC currently reports 99031 deaths. In gen...,real
1,2,States reported 1121 deaths a small rise from ...,real
2,3,Politically Correct Woman (Almost) Uses Pandem...,fake
3,4,#IndiaFightsCorona: We have 1524 #COVID testin...,real
4,5,Populous states can generate large case counts...,real
5,6,"Covid Act Now found ""on average each person in...",real
6,7,If you tested positive for #COVID19 and have n...,real
7,8,Obama Calls Trump’s Coronavirus Response A Cha...,fake
8,9,"???Clearly, the Obama administration did not l...",fake
9,10,Retraction—Hydroxychloroquine or chloroquine w...,fake


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6420 entries, 0 to 6419
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      6420 non-null   int64 
 1   tweet   6420 non-null   object
 2   label   6420 non-null   object
dtypes: int64(1), object(2)
memory usage: 150.6+ KB


In [5]:
sentences = [word_tokenize(text.lower()) for text in tqdm(df['tweet'])]

100%|██████████| 6420/6420 [00:01<00:00, 3575.25it/s]


In [6]:
model_tweets = Word2Vec(sentences, workers=4, size=300, min_count=3, window=5, iter=15)

In [7]:
model_tweets.init_sims()

In [8]:
def get_text_embedding(text):
  result = []
  for word in word_tokenize(text.lower()):
    if word in model_tweets.wv:
      result.append(model_tweets.wv[word])
  if len(result):
    result = np.sum(result, axis = 0)
  else:
    result = np.zeros(300)
  return result

In [9]:
features = [get_text_embedding(text) for text in tqdm(df['tweet'])]

100%|██████████| 6420/6420 [00:05<00:00, 1176.97it/s]


### Support Vector Machines

In [10]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [11]:
X_train, X_test, y_train, y_test = train_test_split(features, df['label'], test_size = 0.3)

In [12]:
svm_model = SVC(kernel='linear').fit(X_train, y_train)
prediction_svm = svm_model.predict(X_test)
print(classification_report(y_test, prediction_svm))

              precision    recall  f1-score   support

        fake       0.93      0.92      0.92       940
        real       0.92      0.93      0.93       986

    accuracy                           0.92      1926
   macro avg       0.92      0.92      0.92      1926
weighted avg       0.92      0.92      0.92      1926



## Random Forest

In [13]:
from sklearn.ensemble import RandomForestClassifier

classifier = RandomForestClassifier(n_estimators=1000, bootstrap = True).fit(X_train, y_train)
prediction_classifier = classifier.predict(X_test)
print(classification_report(y_test, prediction_classifier))

              precision    recall  f1-score   support

        fake       0.95      0.91      0.93       940
        real       0.92      0.95      0.93       986

    accuracy                           0.93      1926
   macro avg       0.93      0.93      0.93      1926
weighted avg       0.93      0.93      0.93      1926



## KNN

In [14]:
from sklearn.neighbors import KNeighborsClassifier

kn = KNeighborsClassifier(n_neighbors=1).fit(X_train, y_train)
prediction_kn = kn.predict(X_test)
print(classification_report(y_test, prediction_kn))

              precision    recall  f1-score   support

        fake       0.92      0.89      0.90       940
        real       0.90      0.92      0.91       986

    accuracy                           0.91      1926
   macro avg       0.91      0.91      0.91      1926
weighted avg       0.91      0.91      0.91      1926



## Logistic Regression + LDA

In [15]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.linear_model import LogisticRegression

lda = LDA(n_components=1)
X_train = lda.fit_transform(X_train, y_train)
X_test = lda.transform(X_test)

model = LogisticRegression().fit(X_train, y_train)
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

        fake       0.92      0.91      0.91       940
        real       0.91      0.92      0.92       986

    accuracy                           0.92      1926
   macro avg       0.92      0.92      0.92      1926
weighted avg       0.92      0.92      0.92      1926



## PyTorch + LSTM

In [16]:
import torch
import torch.nn as nn
import torch.optim as optim

In [17]:
labels = (df['label'] == 'real').astype(int).to_list()

token_lists = [word_tokenize(text.lower()) for text in df.tweet]
max_len = len(max(token_lists, key=len))

In [18]:
def get_word_embedding(tokens, max_len):
    result = []
    for i in range(max_len):
        if i < len(tokens):
            word = tokens[i]
            if word in model_tweets.wv:
                result.append(model_tweets.wv[word])
            else:
                result.append(np.zeros(300))
        else:
            result.append(np.zeros(300))
    return result

In [19]:
features = [get_word_embedding(text, 200) for text in tqdm(token_lists)]

100%|██████████| 6420/6420 [00:04<00:00, 1300.60it/s]


In [20]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.3)

In [21]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.lstm = nn.LSTM(300, 100)
        self.out = nn.Linear(100, 1)

    def forward(self, x):
        embeddings, (shortterm, longterm) = self.lstm(x.transpose(0, 1))
        prediction = torch.sigmoid(self.out(longterm))
        return prediction


net = Net()
print(net)

Net(
  (lstm): LSTM(300, 100)
  (out): Linear(in_features=100, out_features=1, bias=True)
)


In [22]:
in_data = torch.tensor(X_train).float()
targets = torch.tensor(y_train).float()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:210.)
  """Entry point for launching an IPython kernel.


In [23]:
optimizer = optim.SGD(net.parameters(), lr=0.01)
criterion = nn.BCELoss()

In [28]:
def train_one_epoch(n_epochs, in_data, targets, batch_size=16):
  for epoch in range(0, n_epochs + 1):
    for i in tqdm(range(0, in_data.shape[0], batch_size)):
        batch_x = in_data[i:i + batch_size]
        batch_y = targets[i:i + batch_size]
        optimizer.zero_grad()
        output = net(batch_x)
        loss = criterion(output.reshape(-1), batch_y)
        loss.backward()
        optimizer.step()
    print(f'epoch - {epoch}, loss {loss}')

In [29]:
train_one_epoch(50, in_data, targets)

100%|██████████| 281/281 [02:44<00:00,  1.71it/s]


epoch - 0, loss 0.6797469854354858


100%|██████████| 281/281 [02:41<00:00,  1.74it/s]


epoch - 1, loss 0.6799895167350769


100%|██████████| 281/281 [02:40<00:00,  1.75it/s]


...

epoch - 48, loss 0.6799905896186829


100%|██████████| 281/281 [02:42<00:00,  1.73it/s]


epoch - 49, loss 0.6799896955490112


100%|██████████| 281/281 [02:41<00:00,  1.74it/s]

epoch - 50, loss 0.67998868227005


In [30]:
in_data_test = torch.tensor(X_test).float()
targets_test = torch.tensor(y_test).float()

In [31]:
with torch.no_grad():
    output = net(in_data_test).reshape(-1)

result = (output > 0.5) == targets_test
result.sum().item() / len(result)

0.5337487019730011